In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from llm_feature_selector import LLMFeatureSelector

In [3]:
current = pd.read_csv('transformer/CurrentVoltage.csv', index_col='DeviceTimeStamp')
overview = pd.read_csv('transformer/Overview.csv', index_col='DeviceTimeStamp')
power = pd.read_csv('transformer/Power.csv', index_col='DeviceTimeStamp')
power_factor = pd.read_csv('transformer/PowerFactor.csv', index_col='DeviceTimeStamp')
total_power = pd.read_csv('transformer/TotalPower.csv', index_col='DeviceTimeStamp')

In [4]:
current.index = pd.to_datetime(current.index)
overview.index = pd.to_datetime(overview.index)
power.index = pd.to_datetime(power.index)
power_factor.index = pd.to_datetime(power_factor.index)
total_power.index = pd.to_datetime(total_power.index)

In [5]:
transformer = current
for df in [overview, power, power_factor, total_power]:
    transformer = pd.merge(transformer, df, on='DeviceTimeStamp')
transformer = transformer.reset_index()

In [6]:
# X = transformer.drop(['DeviceTimeStamp','MOG_A'],axis=1)
# y = transformer['MOG_A']
# xgb = Xgb.XGBClassifier()
# xgb.fit(X_train,y_train)
# y_pred_xgb = xgb.predict(X_test)
# xgb.score(X_train, y_train)

# xgb_train = round(xgb.score(X_train, y_train) * 100, 2)
# xgb_accuracy = round(accuracy_score(y_pred_xgb, y_test) * 100, 2)

In [7]:
# feature_descriptions = {
#     'VL1': 'Phase Line 1',
#     'VL2': 'Phase Line 2',
#     'VL3': 'Phase Line 3',
#     'IL1': 'Current Line 1',
#     'IL2': 'Current Line 2',
#     'IL3': 'Current Line 3',
#     'VL12': 'Voltage line 1 2',
#     'VL23': 'Voltage line 2 3',
#     'VL31': 'Voltage line 3 1',
#     'INUT': 'Neutral Current',
#     'OTI': 'Oil Temperature Indicator',
#     'WTI': 'Winding Temperature Indicator',
#     'ATI': 'Ambient Temperature Indicator',
#     'OLI': 'Oil Level Indicator',
#     'OTI_A': 'Oil Temperature Indicator Alarm',
# }

In [10]:
# model_name = 'microsoft/phi-4'
model_name = 'meta-llama/Llama-3.3-70B-Instruct'
prompt_template = 'Select the variables from the list that are most relevant for predicting <target_variable>. ' +\
                  'Provide the variables sorted starting with the one with the highest priority. ' +\
                  'All variables: <all_variables>\n' + \
                  '```json\n{"reasoning": "<your reasoning>", "selected_variables": ["variable 1", "variable 2", ..., "variable n"]}\n```'
all_cols = transformer.drop(['DeviceTimeStamp', 'MOG_A'], axis=1).columns.tolist()

In [12]:
prompt = prompt_template.replace('<all_variables>', ', '.join(all_cols))
prompt = prompt.replace('<target_variable>', 'magnetic oil gauge fault in electrical transformers')
print(prompt)

Select the variables from the list that are most relevant for predicting magnetic oil gauge fault in electrical transformers. Provide the variables sorted starting with the one with the highest priority. All variables: VL1, VL2, VL3, IL1, IL2, IL3, VL12, VL23, VL31, INUT, OTI, WTI, ATI, OLI, OTI_A, OTI_T, WL1, WL2, WL3, VAL1, VAL2, VAL3, RVAL1, RVAL2, RVAL3, PFL1, PFL2, PFL3, Avg_PF, Sum_PF, FRQ, THDVL1, THDVL2, THDVL3, THDIL1, THDIL2, THDIL3, MDIL1, MDIL2, MDIL3, KWH, KWH_I, KVARH, KW, KVA, KVAR, MPD, MKVAD
```json
{"reasoning": "<your reasoning>", "selected_variables": ["variable 1", "variable 2", ..., "variable n"]}
```


In [24]:
feat_sel = LLMFeatureSelector(model_name=model_name,
                              feature_names=all_cols,
                              target_variable='magnetic oil gauge fault',
                              prompt_template=prompt_template,
                              topk=3
                             )
output = feat_sel.fit(transformer[all_cols])
output.shape

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{"selected_variables": ["OTI", "OLI", "INUT", "KWH", "KW", "KVA", "KVAR", "MDIL1", "MDIL2", "MDIL3", "THDIL1", "THDIL2", "THDIL3", "THDVL1", "THDVL2", "THDVL3", "VL1", "VL2", "VL3", "IL1", "IL2", "IL3"]}


(3, 48)

In [34]:
sorted_corrs = transformer.drop('DeviceTimeStamp', axis=1).corr()['MOG_A'].abs().sort_values(ascending=False)

(29291, 48)

In [48]:
for rec in output:
    print(rec, sorted_corrs[rec])

OLI 0.20330742879547067
OTI 0.07827422647381119
OTI_A 0.0024881496650764417
OTI_T 0.0030761538542668364
ATI 0.007283277548341243
WTI 0.12523335014341658
